# Intro to mmcv
#### *Author* : Pengxiao Gao (pengxiao.gao@momenta.ai)

In [1]:
import os
import os.path as osp
import mmcv

from mmcv import Config

/home/pengx/miniconda3/envs/torch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Config

In [2]:
cfg = Config(dict(a=1, b=dict(b1=[0, 1])))
print(cfg.b.b1)

[0, 1]


In [3]:
cfg = Config.fromfile('test/config/config_0.py')
print(cfg.pretty_text)

a = 1
b = dict(b1=[0, 1, 2], b2=None)
c = (1, 2)
d = 'string'



For all format configs, some ***predefined variables*** are supported. It will convert the variable in `{{ var }}` with its real value.

Currently, it supports four predefined variables:

`{{ fileDirname }}` - the current opened file’s dirname, e.g. /home/your-username/your-project/folder

`{{ fileBasename }}` - the current opened file’s basename, e.g. file.ext

`{{ fileBasenameNoExtension }}` - the current opened file’s basename with no file extension, e.g. file

`{{ fileExtname }}` - the current opened file’s extension, e.g. .ext

These variable names are referred from VS Code.

Here is one examples of config with predefined variables.

config_a.py

In [4]:
cfg = Config.fromfile('test/config/config_a.py')
print(cfg.pretty_text)

a = 1
b = 'config_a'
c = '.py'



### Inherit from base config


In [5]:
cfg = Config.fromfile('test/config/config_b.py')
print(cfg.pretty_text)

a = 1
b = '.py'
c = 'config_b'
d = 'str'



### Inherit from multiple base configs (the base configs should not contain the same keys)

In [6]:
cfg = Config.fromfile('test/config/config_d.py')
print(cfg.pretty_text)

a = 1
b = 'config_a'
c = '.py'
d = 'str'
e = (1, 2)



### Reference variables from base

In [7]:
cfg = Config.fromfile('test/config/config_e.py')
print(cfg.pretty_text)

a = 1
b = 'config_a'
c = '.py'
d = 'str'
e = (1, 2)
item = dict(a=1, b='config_a')



## Registry

MMCV implements *registry* to manage different modules that share similar functionalities, e.g., backbones, head, and necks, in detectors. Most projects in OpenMMLab use registry to manage modules of datasets and models, such as MMDetection, MMDetection3D, MMClassification, MMEditing, etc.

To manage your modules in the codebase by *Registry*, there are three steps as below.

Create a build method (optional, in most cases you can just use the default one).

Create a `registry`.

Use this `registry` to manage the modules.

Assuming we want to implement a series of Dataset Converter for converting different formats of data to the expected data format. We create a directory as a package named converters. In the package, we first create a file to implement builders


 By using decorator, a mapping between a string and the class (function) is built and maintained by `CONVERTERS` as below

In [8]:
from converters.builder import CONVERTERS
from converters.converter1 import Converter_1
from converters.converter2 import converter_2
print(Converter_1)
print(converter_2)

<class 'converters.converter1.Converter_1'>
<function converter_2 at 0x7f06d3ad8b80>


## Hooking
In computer programming, the term ***hooking*** covers a range of techniques used to ***alter or augment*** the behaviour of an operating system, of applications, or of other software components by intercepting function calls or messages or events passed between software components. Code that handles such intercepted function calls, events or messages is called a hook.

Hooking is used for many purposes, including ***debugging and extending*** functionality. Examples might include ***intercepting*** keyboard or mouse event messages before they reach an application, or intercepting operating system calls in order to ***monitor*** behavior or ***modify*** the function of an application or other component. It is also widely used in benchmarking programs, for example ***fps (frames per seconds) measuring*** in 3D games, where the output and input is done through hooking.

Hooking can also be used by ***malicious*** code. For example, ***rootkits***, pieces of software that try to make themselves invisible by faking the output of API calls that would otherwise reveal their existence, often use hooking techniques. ***Game cheating plug-ins*** are another example 

In [9]:
def hook(d):
   print(d)

def add(a,b,c,hook_fn=None):
   sum1=a+b
   if hook_fn is not None:
       hook_fn(sum1)
       return sum1+c

# 调用
add(1,2,3,hook)

3


6

### Hook in mmcv
In mmcv Hooks can be registered into the Runner.

## Runner
The runner class is designed to manage the training. It eases the training process with less code demanded from users while staying flexible and configurable. 
The process of using the Runner can be divided into 4 steps.

1. **Initialisation** of the Runner object
2. **Registering** each type of Hook to the Runner
3. Calling the Runner's **resume** or **load_checkpoint** method to load the weights
4. Running the given **workflow**, which actually starts the workflow

### Initialisation of the Runner
BaseRunner for reuse

### Register Hook
`register_training_hooks()`

### Resume and `load_checkpoint`
The resume method is used to load weights when training is stopped and then resumed, while load_checkpoint only loads pre-trained weights

### Run
The run method is actually openning the workflow, and because the Epoch and Iter mode processes are different, they are implemented in their respective subclasses.

#### EpochBasedRunner
EpochBasedRunner is a workflow with epochs, e.g. setting workflow = [('train', 2), ('val', 1)] means iteratively training 2 epochs and then validating 1 epoch. The MMDetection target detection framework uses the EpochBasedRunner by default.

### IterBasedRunner

Since the IterBasedRunner mode does not have the concept of epoch, the run method of the IterBasedRunner has been changed slightly

The workflow termination condition is no longer epoch, but iter
Hook's lifecycle methods also do not involve epoch, they are all iter-related methods
Although the epoch parameter is not needed in this class, it is still needed in many scenarios, so the IterLoader is used to get the epoch parameter.